In [96]:
from google.colab import files #import utils.py
files.upload()

{}

In [97]:
#packages
!pip install tensorboardX #dependancy for Logger
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import utils
from utils import Logger


In [0]:
train_loader = torch.utils.data.DataLoader(
 dsets.MNIST('/content', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize( (0.1307,), (0.3081,))])),
  batch_size=100, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  dsets.MNIST('/content', train=False, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=100, shuffle=True)

In [0]:
img_size=28
class Discriminator(nn.Module):
  def __init__(self,hiddenlayer0_out,hiddenlayer1_out,hiddenlayer2_out):
    super(Discriminator,self).__init__()
    in_n=28*28
    self.hidden0=nn.Sequential(
        nn.Linear(in_n,hiddenlayer0_out),
        nn.LeakyReLU(0.2),
        nn.Dropout2d(0.3)
    )
    self.hidden1=nn.Sequential(
         nn.Linear(hiddenlayer0_out,hiddenlayer1_out),
        nn.LeakyReLU(0.2),
        nn.Dropout2d(0.3)
    )
    self.hidden2=nn.Sequential(
         nn.Linear(hiddenlayer1_out,hiddenlayer2_out),
        nn.LeakyReLU(0.2),
        nn.Dropout2d(0.3)
    )
    self.outlayer=nn.Sequential(
        nn.Linear(hiddenlayer2_out,1),
        nn.Sigmoid()
    )

  def forward(self,x):
    out=self.hidden0(x)
    out=self.hidden1(out)
    out=self.hidden2(out)
    out=self.outlayer(out)
    return out


In [0]:
img_size=28
class Generator(nn.Module):
  def __init__(self,hiddenlayer0_out,hiddenlayer1_out,hiddenlayer2_out):
    super(Generator,self).__init__()
    in_n=100
    out_n=28*28
    self.hidden0=nn.Sequential(
        nn.Linear(in_n,hiddenlayer0_out),
        nn.LeakyReLU(0.2),
    )
    self.hidden1=nn.Sequential(
         nn.Linear(hiddenlayer0_out,hiddenlayer1_out),
        nn.LeakyReLU(0.2),
    )
    self.hidden2=nn.Sequential(
         nn.Linear(hiddenlayer1_out,hiddenlayer2_out),
        nn.LeakyReLU(0.2),
    )
    self.outlayer=nn.Sequential(
        nn.Linear(hiddenlayer2_out,out_n),
        nn.Tanh()
    )

  def forward(self,x):
    out=self.hidden0(x)
    out=self.hidden1(out)
    out=self.hidden2(out)
    out=self.outlayer(out)
    return out


In [0]:

def noise(size):
  n=Variable(torch.randn(size,100).cuda()) #torch.randn returns normalized random numbers between 0 and 1 (100 is our batch size , "size" variable is our noise vector size)
  return n
def target_zeros(size): 
  n=Variable(torch.zeros(size,1).cuda())
  return n
def target_ones(size): 
  n=Variable(torch.ones(size,1).cuda())
  return n


In [113]:
disc_model=Discriminator(1024,512,256)
disc_model.cuda()
gen_model=Generator(256,512,1024)
gen_model.cuda()  
disc_optimizer=optim.Adam(disc_model.parameters(),lr=0.0002)
gen_optimizer=optim.Adam(gen_model.parameters(),lr=0.0002)
loss=nn.BCELoss() # in torch we use minimze function(even though we had to use gradient ascent), BCE loss has a negitive sign which does this job for us

RuntimeError: ignored

In [0]:
size=100
def train_disc(optimizer,real,fake) :# the discriminator takes a real image and a fake image and predicts output
  optimizer.zero_grad()
  
  predicted_real=disc_model(real)
  error_real=loss(predicted_real,target_ones(size)) #compare with ones and calculate loss gradients then update weights
  error_real.backward()
  predicted_fake=disc_model(fake)
  error_fake=loss(predicted_fake,target_zeros(size)) # compare with zeros and calculate loss gradients then update weights
  optimizer.step() # update weights
  return (error_real+error_fake),predicted_real,predicted_fake # see discriminator's formula for real+fake errors

In [0]:
def train_gen(optimizer,fake_image):
  optimizer.zero_grad()
  N=fake_image.size(0)
  
  prediction=disc_model(fake_image)
  error=loss(prediction,target_ones(N))# get max 1 s for generator
  error.backward()
  
  optimizer.step()
  return error
  

In [111]:
%xmode plain

logger = Logger(model_name='VGAN', data_name='MNIST')
epoch=100
for epochs in range(epoch):
  j=0
  for nbatch,(images,labels) in enumerate(train_loader):
    j=j+1
    images=(images.view(-1,28*28))
    real_image=Variable(images.cuda())
    fake_image=gen_model(noise(size)).detach()
    
    d_error,real,fake=train_disc(disc_optimizer,real_image,fake_image) #train discriminator
    
    fake_image=gen_model(noise(size))
    g_error=train_gen(gen_optimizer,fake_image) # train generator
    
    logger.log(d_error, g_error, epoch, nbatch, 100)
    if j%1000==0:
      test=optimizer(noise(16))
      test=test.view(-1,1,28,28)
      logger.log_images(test, 16,epochs, nbatch, 100)
      logger.display_status(epochs, epoch, nbatch, 100,d_error, g_error, real, fake)

Exception reporting mode: Plain


RuntimeError: ignored